<a href="https://colab.research.google.com/github/elliemci/chatbots/blob/main/msg_intent_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NLP classification integrated into a telegram chatbot that classifyes any user message according to a set of intents

In [1]:
# install Python transformers development version
!pip install transformers[sentencepiece] pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.4 MB/s eta 0:00:00
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.14.0-py3-none-any.whl size=215250 sha256=83fa8b51f7dbdcf1c4cf798e7a3d43030cfe04b125a8a7d9804c6f05ab294268
  Stored in directory: /root/.cache/pip/wheels/25/51/2d/24b40a366c85c37928d5aa36ddf257e5a79fad25e1ecd11b2c
Successfully built pyTelegramBotAPI


In [3]:
# import pyTelegramBotAPI and pipline class
import telebot

from transformers import pipeline

### Set the chatbot token

In [4]:
import os
from getpass import getpass

# text /newbot to the BotFather in the Telegram Chat choose a bot's name and username
os.environ['TELEGRAM_BOT_TOKEN'] = getpass('Enter your bot token: ')
TOKEN = os.getenv('TELEGRAM_BOT_TOKEN')

# create the bot an instance of TeleBot when set the token
bot = telebot.TeleBot(TOKEN)

Enter your bot token: ··········


In [6]:
# instantiate piplenine class passing the "zero-shot-classification" task, i.e. classification
# is done without any trainig specific to classification in hand, default transformer model BART
classifier = pipeline("zero-shot-classification")
possible_intents = ["food", "drinks", "deserts", "sports", "movies", "music", "news", "fun", "social media", "focus"]

# define a dictionary to store the state of the conversation
conversation_state = {}

# function that handles all incoming messages, filter always returns True
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    global conversation_state
    # get the chat id
    chat_id = message.chat.id
    # check if the chat id is in the conversation state dictionary
    if chat_id not in conversation_state:
      conversation_state[chat_id] = "start"
      print(f"conversation state {conversation_state}")
    # get the current state of the conversation
    state = conversation_state[chat_id]

    # check the content of the message and classify it
    if state == "start":
      bot.send_message(chat_id, "Hi, how can I help you?")
      conversation_state[chat_id] = "waiting_for_input"
    elif state == "waiting_for_input":
      # get user's message intent
      intent = classifier(message.text,  possible_intents)
      # get the label of the intent
      label = intent["labels"][0]
      # get the score of the intent
      score = intent["scores"][0]
      # reply with the label if score is higher than 0.5, otherwise reply with default message
      if score > 0.5:
        bot.send_message(chat_id, f"Your intent is {label}")
      else:
        bot.send_message(chat_id, "I'm sorry, I didn't understand your intent.")
        conversation_state[chat_id] = "waiting_for_input"

# start the bot
bot.polling()

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


chat id 6432666165
conversation state {6432666165: 'start'}
conversation state {6432666165: 'waiting_for_input'}
chat id 6432666165
classified intent {'sequence': 'I want to watch CNN', 'labels': ['news', 'focus', 'fun', 'social media', 'movies', 'deserts', 'sports', 'drinks', 'food', 'music'], 'scores': [0.4678856432437897, 0.28161299228668213, 0.1875370889902115, 0.02514306828379631, 0.009182273410260677, 0.007911425083875656, 0.007125344593077898, 0.005344109609723091, 0.0047874595038592815, 0.0034705596044659615]}
labels news with scores 0.4678856432437897
chat id 6432666165
classified intent {'sequence': 'I want to listen to NPR   All  things considered', 'labels': ['music', 'focus', 'fun', 'news', 'deserts', 'social media', 'drinks', 'sports', 'food', 'movies'], 'scores': [0.44553816318511963, 0.23911263048648834, 0.13768354058265686, 0.1365867257118225, 0.009137795306742191, 0.007187732961028814, 0.007005343679338694, 0.006874744780361652, 0.006471486762166023, 0.004401822108775

Chatbot that receives a message from the user and classifies the message acording to set of intents